In [340]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

#### Reading .csv files to analize data

In [341]:
master_data = pd.read_csv('data/master.csv', index_col = 'playerID', usecols = [0, 13, 14])
master_data.head()

,nameFirst,nameLast
playerID,,
aardsda01,David,Aardsma
aaronha01,Hank,Aaron
aaronto01,Tommie,Aaron
aasedo01,Don,Aase
abadan01,Andy,Abad


In [342]:
salary_data = pd.read_csv('data/salaries.csv', index_col = 'playerID', usecols = [3, 4])
salary_data.head()

,salary
playerID,
barkele01,870000
bedrost01,550000
benedbr01,545000
campri01,633333
ceronri01,625000


In [343]:
batting_data = pd.read_csv('data/batting.csv', index_col = 'playerID', usecols = [0, 1, 6, 8, 15, 18, 20])
batting_data.head()

,yearID,AB,H,BB,HBP,SF
playerID,,,,,,
abercda01,1871,4.0,0.0,0.0,NaN,NaN
addybo01,1871,118.0,32.0,4.0,NaN,NaN
allisar01,1871,137.0,40.0,2.0,NaN,NaN
allisdo01,1871,133.0,44.0,0.0,NaN,NaN
ansonca01,1871,120.0,39.0,2.0,NaN,NaN


In [344]:
batting_no_nan = batting_data.fillna(0)
batting_no_nan.head()

,yearID,AB,H,BB,HBP,SF
playerID,,,,,,
abercda01,1871,4.0,0.0,0.0,0.0,0.0
addybo01,1871,118.0,32.0,4.0,0.0,0.0
allisar01,1871,137.0,40.0,2.0,0.0,0.0
allisdo01,1871,133.0,44.0,0.0,0.0,0.0
ansonca01,1871,120.0,39.0,2.0,0.0,0.0


In [345]:
fielding_data = pd.read_csv('data/fielding.csv', index_col = 'playerID', usecols = [0, 5])
fielding_data.head()

,POS
playerID,
abercda01,SS
addybo01,2B
addybo01,SS
allisar01,2B
allisar01,OF


In [346]:
name_position = master_data.merge(fielding_data, left_index=True, right_index=True)
name_position.head()

,nameFirst,nameLast,POS
playerID,,,
aardsda01,David,Aardsma,P
aardsda01,David,Aardsma,P
aardsda01,David,Aardsma,P
aardsda01,David,Aardsma,P
aardsda01,David,Aardsma,P


In [347]:
name_position_salary = name_position.merge(salary_data, left_index=True, right_index=True)
name_position_salary.head()

,nameFirst,nameLast,POS,salary
playerID,,,,
aardsda01,David,Aardsma,P,300000
aardsda01,David,Aardsma,P,387500
aardsda01,David,Aardsma,P,403250
aardsda01,David,Aardsma,P,419000
aardsda01,David,Aardsma,P,2750000


In [348]:
total_info = name_position_salary.merge(batting_no_nan, left_index=True, right_index=True)
total_info.head()

,nameFirst,nameLast,POS,salary,yearID,AB,H,BB,HBP,SF
playerID,,,,,,,,,,
aardsda01,David,Aardsma,P,300000,2004,0.0,0.0,0.0,0.0,0.0
aardsda01,David,Aardsma,P,300000,2006,2.0,0.0,0.0,0.0,0.0
aardsda01,David,Aardsma,P,300000,2007,0.0,0.0,0.0,0.0,0.0
aardsda01,David,Aardsma,P,300000,2008,1.0,0.0,0.0,0.0,0.0
aardsda01,David,Aardsma,P,300000,2009,0.0,0.0,0.0,0.0,0.0


### OBP = (H + BB + HBP) / (AB + BB + HBP + SF)

H - Hits;    BB - Base on Balls;    HBP - Hit by Pitch;    SF - Sacrifice flies

In [361]:
total_info['OBP'] = (total_info['H']+ total_info['BB']+ total_info['HBP']) / (total_info['AB']+ total_info['BB']+ total_info['HBP'] + total_info['SF'])
total_info.head()

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,nameFirst,nameLast,POS,salary,yearID,AB,H,BB,HBP,SF,OBP
playerID,,,,,,,,,,,
burriem01,Emmanuel,Burriss,2B,410000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,LF,410000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,LF,625000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,SS,404000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,SS,410000,2015,3.0,2.0,2.0,0.0,0.0,0.8


### Filtering DataFrame 
For the data analysis there's no need to have players with no salary and players whose OBP is either 1 or 0.

In [366]:
total_info = total_info[total_info['OBP'] != 0]

In [367]:
total_info = total_info[total_info['OBP'] != 1]

In [368]:
total_info = total_info[total_info['salary'] != 0]
total_info.head()

,nameFirst,nameLast,POS,salary,yearID,AB,H,BB,HBP,SF,OBP
playerID,,,,,,,,,,,
burriem01,Emmanuel,Burriss,2B,410000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,LF,410000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,LF,625000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,SS,404000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,SS,410000,2015,3.0,2.0,2.0,0.0,0.0,0.8


In [365]:
total_info = total_info.dropna()
total_info.head()

,nameFirst,nameLast,POS,salary,yearID,AB,H,BB,HBP,SF,OBP
playerID,,,,,,,,,,,
burriem01,Emmanuel,Burriss,2B,410000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,LF,410000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,LF,625000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,SS,404000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,SS,410000,2015,3.0,2.0,2.0,0.0,0.0,0.8


##### We will consider players that played after 2014

In [372]:
total_info = total_info.query('yearID>2014')

In [373]:
#total_info = total_info.sort(['OBP'], ascending = False)
total_info = total_info.sort_values(by = 'OBP', ascending = False)
total_info

,nameFirst,nameLast,POS,salary,yearID,AB,H,BB,HBP,SF,OBP
playerID,,,,,,,,,,,
burriem01,Emmanuel,Burriss,2B,410000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,3B,625000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,LF,410000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,1B,404000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,1B,410000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,1B,625000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,LF,404000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,RF,625000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,RF,404000,2015,3.0,2.0,2.0,0.0,0.0,0.800000


#### The roster doesn't include Designated Hitter (DH) and Outfielder (OF) positions

In [374]:
total_info = total_info[total_info['POS'] != 'DH']
total_info = total_info[total_info['POS'] != 'OF']
total_info

,nameFirst,nameLast,POS,salary,yearID,AB,H,BB,HBP,SF,OBP
playerID,,,,,,,,,,,
burriem01,Emmanuel,Burriss,2B,410000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,3B,625000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,LF,410000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,1B,404000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,1B,410000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,1B,625000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,LF,404000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,RF,625000,2015,3.0,2.0,2.0,0.0,0.0,0.800000
burriem01,Emmanuel,Burriss,RF,404000,2015,3.0,2.0,2.0,0.0,0.0,0.800000


In [375]:
total_info = total_info.drop_duplicates()
# total = total_info.groupby('POS')
total_info.head()

,nameFirst,nameLast,POS,salary,yearID,AB,H,BB,HBP,SF,OBP
playerID,,,,,,,,,,,
burriem01,Emmanuel,Burriss,2B,410000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,3B,625000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,LF,410000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,1B,404000,2015,3.0,2.0,2.0,0.0,0.0,0.8
burriem01,Emmanuel,Burriss,1B,410000,2015,3.0,2.0,2.0,0.0,0.0,0.8


#### Below is the minimum salary and maximum OBP for each position

In [376]:
total_info.groupby('POS').salary.min()

POS
1B    165574
2B    170000
3B    165574
C     210000
CF    200000
LF    165574
P     150000
RF    165574
SS    170000
Name: salary, dtype: int64

In [377]:
total_info.groupby('POS').OBP.max()

POS
1B    0.800000
2B    0.800000
3B    0.800000
C     0.444444
CF    0.460245
LF    0.800000
P     0.666667
RF    0.800000
SS    0.800000
Name: OBP, dtype: float64